In [16]:
import numpy as np
import pandas as pd
import matplotlib as olt
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

2.1.0


In [17]:
imdb = keras.datasets.imdb
# 设置数据中词表的个数,前10000个保留，后面的作为特殊字符处理
vocab_size = 10000
# 词表的index从几开始算
index_from = 3
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
                                                       num_words = vocab_size,
                                                       index_from = index_from)


In [18]:
# 打印词表
word_index = imdb.get_word_index()
print(len(word_index))
# key-value形式，key:词语，value：ID
# print(word_index)

88584


In [19]:
# 因为index_from = 3，所以需要下面的操作
word_index = {k:(v+3) for k, v in word_index.items()}

In [20]:
# 存储特殊字符
word_index['<PAD>'] = 0
word_index['<START>'] = 1
word_index['<UNK>'] = 2  # 找不到字符返回 UNK
word_index['<END>'] = 3

# 生成反向索引  ID --> word
# 定义函数，使train_data反向映射回来，看他具体的文本是什么。
reverse_word_index = dict([(value, key) for key, value in word_index.items()])
def decode_review(text_ids):
    return ' '.join([reverse_word_index.get(word_id, '<NUK>') for word_id in text_ids])

decode_review(train_data[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

In [21]:
# 给数据做补全
max_length = 500  # 长度低于500做补全，高于500做截断

train_data = keras.preprocessing.sequence.pad_sequences(
    train_data, # list of list
    value = word_index['<PAD>'], # 填充的值
    padding = 'post', # 填充顺序，post：放在句子后面， pre：放在句子后面
    maxlen = max_length
)

test_data = keras.preprocessing.sequence.pad_sequences(
    test_data,
    value = word_index['<PAD>'],
    padding = 'post',
    maxlen = max_length
)

print(train_data[0])

[   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256
    4    2    7 3766    5  723   36   71   43  530  476   26  400  317
   46    7    4    2 1029   13  104   88    4  381   15  297   98   32
 2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
   26  480    5  144   30 5535   18   51   36   28  224   92   25  104
    4 

单层单向LSTM

In [ ]:
# 定义模型
embedding_dim = 16 # 每一个word将其embedding成长度为16的向量
batch_size = 28

single_lstm_model = keras.models.Sequential([
    # 1、define matrix: [vocab_size, embedding_dim]
    # 2、将样本 word[4.2, 7.6, 5.5,..., 9.1] 中对应的数字去步骤1中的矩阵查，
    #    每个数字对应一个向量，则word转换为max_length(做了padding) * embedding_dim的矩阵
    # 3、batch_size * max_length * embedding_dim
    keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    
    keras.layers.LSTM(units = 64, return_sequences = False), # 只取最后一层的输出
    keras.layers.Dense(64, activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid')
])

single_lstm_model.summary()
single_lstm_model.compile(loss = 'binary_crossentropy', optimizer = 'adam',
                          metrics = ['accuracy'])

In [ ]:
history = single_lstm_model.fit(train_data, train_labels,
                                epochs = 10,
                                batch_size = batch_size,
                                validation_split = 0.2)

In [ ]:
def plot_learning_curve(history, label, epochs, min_value, max_value):
    data = {}
    data[label] = history.history[label]
    data['val_' + label] = history.history['val_' + label]
    pd.DataFrame(data).plot(figsize = (8, 5))
    plt.grid()
    plt.axis([0, epochs, min_value, max_value])
    plt.show()
    
plot_learning_curve(history, "accuracy", 10, 0.5, 1)
plot_learning_curve(history, "loss", 10, 0, 1)

In [ ]:
single_lstm_model.evaluate(test_data, test_labels, batch_size = batch_size)

多层双向LSTM

In [22]:
embedding_dim = 16
batch_size = 128

model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim,
                           input_length = max_length),
    keras.layers.Bidirectional(
        keras.layers.LSTM(
            units = 64, return_sequences = True)),  # 注意这里要变成True，看笔记的结构图
    keras.layers.Bidirectional(
        keras.layers.LSTM(
            units = 64, return_sequences = False)),
    keras.layers.Dense(64, activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid')
])

model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 16)           160000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 500, 128)          41472     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 308,609
Trainable params: 308,609
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(train_data, train_labels, epochs = 5, 
                    batch_size = batch_size, validation_split = 0.2)
def plot_learning_curve(history, label, epochs, min_value, max_value):
    data = {}
    data[label] = history.history[label]
    data['val_' + label] = history.history['val_' + label]
    pd.DataFrame(data).plot(figsize = (8, 5))
    plt.grid()
    plt.axis([0, epochs, min_value, max_value])
    plt.show()
    
plot_learning_curve(history, "accuracy", 10, 0.5, 1)
plot_learning_curve(history, "loss", 10, 0, 1)